In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import numpy as np
from IPython.display import HTML
from utils import *

np.random.seed(0)
%matplotlib inline

Pedestrians' average speed: 35 by x-coordinate and 30 by y-coordinate.

So let's assume pedestrian takes the whole 30x30 block. That minimize our 1920x1080 field to 64x36 on which we will be calculate robot's path

In [2]:
STEP = 30

In [3]:
paths = np.load('src/all_not_ext_paths_with_ids.npy')
ped_time = (paths[:, 1] / 20).astype(int)
ped_col = (paths[:, 2] / STEP).astype(int)
ped_row = (paths[:, 3] / STEP).astype(int)

In [4]:
obs = Image.open('src/Images/obstacles.png')
obs = (~np.array(obs).sum(axis=2).astype(bool)).astype(int)

grid = np.zeros((int(FRAME_HEIGHT / STEP), int(FRAME_WIDTH / STEP)), dtype=int)

for i in range(grid.shape[0]):
    for j in range(grid.shape[1]):
        x1, x2 = i * STEP, (i + 1) * STEP
        y1, y2 = j * STEP, (j + 1) * STEP
        if obs[x1 : x2, y1 : y2].sum() > STEP * STEP / 2:
            grid[i, j] = 1

In [5]:
row_start = 30 # from 0 to 35
col_start = 10 # from 0 to 63
row_goal = 10 # from 0 to 35
col_goal = 40 # from 0 to 63
start_time = 1 # from 0 to 6000

assert grid[row_start, col_start] == 0
assert grid[row_goal, col_goal] == 0

t = ped_time == time
ped_row_time = ped_row[t]
ped_col_time = ped_col[t]
ind = (ped_row_time == row_start) * (ped_col_time == row_goal)
assert not ind.any()

In [6]:
# dummy robot
row_dir = np.sign(row_goal - row_start)
col_dir = np.sign(col_goal - col_start)

time = start_time

row_cur, col_cur = row_start, col_start

while row_cur != row_goal or col_cur != col_goal:
    time += 1
    if row_cur != row_goal:
        row_next = row_cur + row_dir
    if col_cur != col_goal:
        col_next = col_cur + col_dir
    
    if ((ped_row[ped_time == time] == row_next) * (ped_col[ped_time == time] == col_next)).any():
        continue
    
    row_cur, col_cur = row_next, col_next
    
time - start_time

30

In [7]:
def get_grid(time, row, col):
    t = ped_time == time
    ped_row_time = ped_row[t]
    ped_col_time = ped_col[t]
        
    im = Image.open(get_framename(time * 20))
    
    draw = ImageDraw.Draw(im)
    for i in range(int(FRAME_HEIGHT / STEP)):
        for j in range(int(FRAME_WIDTH / STEP)):
            if grid[i,j] == 1:
                continue
            ind = (ped_row_time == i) * (ped_col_time == j)
            if ind.any():
                row1, row2 = i * STEP, (i + 1) * STEP
                col1, col2 = j * STEP, (j + 1) * STEP
                draw.rectangle((col1+1, row1+1, col2, row2), fill='blue')
                
    row1, row2 = row * STEP, (row + 1) * STEP
    col1, col2 = col * STEP, (col + 1) * STEP
    draw.rectangle((col1+1, row1+1, col2, row2), fill='red')
                
    return im

In [8]:
def neighbor_cells(i, j, r):
    res = [(x, j-r) for x in range(i-r, i+r+1)]
    res += [(x, j+r) for x in range(i-r, i+r+1)]
    res += [(j-r, y) for y in range(j-r+1, j+r)]
    res += [(j+r, y) for y in range(j-r+1, j+r)]
    return res

In [13]:
def update_cells(pf, row, col, time, block, radius):
    for r in range(radius+1):
        t = ped_time == time + r
        ped_row_time = ped_row[t]
        ped_col_time = ped_col[t]
        
        for i, j in neighbor_cells(row, col, 1 if r == 0 else r):
            ind = (ped_row_time == i) * (ped_col_time == j)
            if ind.any():
                pf.update_cell(int(i), int(j), -1 if block else 0)

In [20]:
from d_star.d_star import DStar

trace = []
pf = DStar(row_start, col_start, row_goal, col_goal)

for i, row in enumerate(grid):
    for j, cell in enumerate(row):
        if cell == 1:
            pf.update_cell(i, j, -1)
            
pf.replan()

time = start_time

row_cur, col_cur = row_start, col_start

trace.append(get_grid(time, row_cur, col_cur))

while row_cur != row_goal or col_cur != col_goal:
    pf.update_start(row_cur, col_cur)
    update_cells(pf, row_cur, col_cur, time, block=True, radius=1)

    time += 1
    
    if not pf.replan():
        print('waiting')
        update_cells(pf, row_prev, col_prev, time-1, block=False, radius=1)
        trace.append(get_grid(time, row_cur, col_cur))
        continue
    
    row_prev, col_prev = row_cur, col_cur
    row_cur, col_cur = pf.get_path()[1].x, pf.get_path()[1].y
    trace.append(get_grid(time, row_cur, col_cur))
    
    update_cells(pf, row_prev, col_prev, time-1, block=False, radius=1)
    
time - start_time

30

In [22]:
%%capture

fig = plt.figure(figsize=(12, 7))
ax = fig.add_subplot(111)
ax.set_axis_off()
fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)  # removes white border
imgs = [(ax.imshow(img), ax.annotate('',(5,5))) for img in trace] 

anim = animation.ArtistAnimation(fig, imgs, interval=200, repeat_delay=2000, blit=False)

In [23]:
rc('animation', html='html5')
anim

In [23]:
anim.save('src/Images/anim.gif', writer='imagemagick', fps=5)